In [3]:
import cv2
import pickle
import numpy as np
import json
import os
from glob import glob
import torch
from PIL import Image
from torch import nn
from torchvision import transforms
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch import optim
from tqdm.auto import tqdm
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sys import getsizeof
from pympler import asizeof
import pandas as pd
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [1]:
import cv2
import mediapipe as mp
import numpy as np
# 카메라 캡처를 초기화합니다.
cap = cv2.VideoCapture(0)  # 0은 기본 카메라를 의미합니다.
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
# 녹화 상태를 나타내는 변수를 초기화합니다.
recording = False
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 비디오 코덱을 설정합니다.
out = None

while cap.isOpened():
    
    with mp_hands.Hands(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
        # 카메라에서 프레임을 읽어옵니다.
        ret, frame = cap.read()
        success, image = cap.read()
        frame = cv2.flip(frame, 1)
    #     # 프레임을 화면에 표시합니다.
        cv2.imshow('Video', frame)

        # 's' 키를 누르면 녹화를 시작합니다.
        if cv2.waitKey(1) & 0xFF == ord('s'):
            recording = True
            out = cv2.VideoWriter('output.avi', fourcc, 20.0, (frame.shape[1], frame.shape[0]))  # 출력 파일을 설정합니다.

        # 녹화 중일 때 프레임을 저장합니다.
        if recording:
            out.write(frame)

        # 't' 키를 누르면 녹화를 중지하고 저장 파일을 닫습니다.
        if cv2.waitKey(1) & 0xFF == ord('t'):
            if recording:
                recording = False
                out.release()  # 녹화 파일을 닫습니다.
                print("녹화가 종료되었습니다.")
        
        # 'q' 키를 누르면 루프를 종료합니다.
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# 카메라 캡처를 해제하고 창을 닫습니다.
cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = None
recording = False

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        continue

    # 화면에 손 감지된 랜드마크 표시
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    if results.multi_hand_landmarks:
        for landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)
            
    frame = cv2.flip(frame, 1)
    cv2.imshow('Video', frame)

    # 녹화 시작
    if cv2.waitKey(1) & 0xFF == ord('s'):
        recording = True
        out = cv2.VideoWriter('output.avi', fourcc, 20.0, (frame.shape[1], frame.shape[0]))

    # 녹화 중지 및 파일 저장
    if cv2.waitKey(1) & 0xFF == ord('t'):
        if recording:
            recording = False
            out.release()
            print("녹화가 종료되었습니다.")

    # 녹화 중일 때 프레임 저장
    if recording:
        out.write(frame)

    # 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [7]:
import cv2
import mediapipe as mp
import numpy as np

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = None
recording = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    # 손 감지 및 랜드마크 추출
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 손 랜드마크만 추출하여 이진 이미지 생성 (손의 랜드마크만 흰색으로, 배경은 검은색으로)
            hand_landmarks_image = np.zeros_like(frame)
            for lm in hand_landmarks.landmark:
                x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                cv2.circle(hand_landmarks_image, (x, y), 5, (255, 255, 255), -1)

    cv2.imshow('Video', frame)

    # 녹화 시작
    if cv2.waitKey(1) & 0xFF == ord('s'):
        recording = True
        out = cv2.VideoWriter('output.avi', fourcc, 20.0, (frame.shape[1], frame.shape[0]))

    # 녹화 중지 및 파일 저장
    if cv2.waitKey(1) & 0xFF == ord('t'):
        if recording:
            recording = False
            out.release()
            print("녹화가 종료되었습니다.")

    # 녹화 중일 때 프레임 저장
    if recording:
        out.write(hand_landmarks_image)

    # 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# 안녕하세요, 점심, 빵, 먹다, 가다

In [16]:
import cv2
import mediapipe as mp
import numpy as np

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = None
recording = False

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    if not ret:
        continue

    # 손 감지 및 랜드마크 추출
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 손 랜드마크만 추출하여 이진 이미지 생성 (손의 랜드마크만 흰색으로, 배경은 검은색으로)
            hand_landmarks_image = np.zeros_like(frame)
            for lm in hand_landmarks.landmark:
                x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                cv2.circle(hand_landmarks_image, (x, y), 5, (255, 255, 255), -1)

            # 랜드마크 간의 선 그리기
            connections = mp_hands.HAND_CONNECTIONS
            for connection in connections:
                connection_point_a = tuple(
                    np.multiply([hand_landmarks.landmark[connection[0]].x, hand_landmarks.landmark[connection[0]].y],
                                [frame.shape[1], frame.shape[0]]).astype(int))
                connection_point_b = tuple(
                    np.multiply([hand_landmarks.landmark[connection[1]].x, hand_landmarks.landmark[connection[1]].y],
                                [frame.shape[1], frame.shape[0]]).astype(int))
                cv2.line(hand_landmarks_image, connection_point_a, connection_point_b, (255, 255, 255), 2)
    
    cv2.imshow('Video', hand_landmarks_image)

    # 녹화 시작
    if cv2.waitKey(1) & 0xFF == ord('s'):
        recording = True
        out = cv2.VideoWriter('output.avi', fourcc, 20.0, (frame.shape[1], frame.shape[0]))

    # 녹화 중지 및 파일 저장
    if cv2.waitKey(1) & 0xFF == ord('t'):
        if recording:
            recording = False
            out.release()
            print("녹화가 종료되었습니다.")

    # 녹화 중일 때 프레임 저장
    if recording:
        out.write(hand_landmarks_image)

    # 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [34]:
import cv2
import mediapipe as mp
import numpy as np

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = None
recording = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    # 손 감지 및 랜드마크 추출
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # 검은색 배경 생성
    hand_landmarks_image = np.zeros_like(frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 손 랜드마크를 화면에 표시
            mp_drawing.draw_landmarks(hand_landmarks_image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                                      landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2),
                                      connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2))

    cv2.imshow('Video', hand_landmarks_image)

    # 녹화 시작
    if cv2.waitKey(1) & 0xFF == ord('s'):
        recording = True
        out = cv2.VideoWriter('output.avi', fourcc, 20.0, (frame.shape[1], frame.shape[0]))

    # 녹화 중지 및 파일 저장
    if cv2.waitKey(1) & 0xFF == ord('t'):
        if recording:
            recording = False
            out.release()
            print("녹화가 종료되었습니다.")

    # 녹화 중일 때 프레임 저장
    if recording:
        out.write(hand_landmarks_image)

    # 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


녹화가 종료되었습니다.


In [21]:
import cv2

# AVI 파일 경로 설정
avi_file_path = r'C:/PlayData/sign_remaster/Sign_Language_Remaster/data/hello/hello_9.avi'

# AVI 파일 열기
video_capture = cv2.VideoCapture(avi_file_path)

# 영상의 프레임 수, 높이, 너비, 채널 수 확인
frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
channels = 3  # AVI 파일의 경우 RGB 채널이므로 3

print("프레임 수:", frame_count)
print("높이:", frame_height)
print("너비:", frame_width)
print("채널 수:", channels)

# AVI 파일 닫기
video_capture.release()


프레임 수: 122
높이: 1080
너비: 1920
채널 수: 3


In [51]:
import cv2
import numpy as np
import glob
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf

# 데이터 폴더 경로
data_folder = r'C:\PlayData\sign_remaster\Sign_Language_Remaster\data'

# 데이터 전처리 함수
def adjust_frames(video_frames, target_frame_count, target_height, target_width):
    total_frames = len(video_frames)
    if total_frames >= target_frame_count:
        # 동영상의 프레임 수가 목표 프레임 수보다 크거나 같으면 샘플링하여 조정
        step = total_frames // target_frame_count
        adjusted_frames = video_frames[::step][:target_frame_count]
    else:
        # 동영상의 프레임 수가 목표 프레임 수보다 작으면 선형 보간하여 조정
        interpolation_factor = total_frames / target_frame_count
        indices = np.arange(0, total_frames, interpolation_factor)[:target_frame_count]
        adjusted_frames = np.array([cv2.resize(frame, (target_width, target_height)) for frame in video_frames])[indices.astype(int)]
    return np.array(adjusted_frames)


# load_and_preprocess_data 함수 수정
def load_and_preprocess_data(file_paths, labels, target_frame_count, target_height, target_width):
    frames = []
    for file_path in file_paths:
        cap = cv2.VideoCapture(file_path)
        frame_list = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (target_width, target_height))  # 모든 프레임을 동일한 크기로 조정
            frame_list.append(frame)
        cap.release()
        # 프레임 수 조정
        adjusted_frames = adjust_frames(frame_list, target_frame_count,128,128)
        frames.append(adjusted_frames)
    data = np.array(frames)
    labels = np.array(labels)
    data = data / 255.0
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    onehot_encoder = OneHotEncoder(sparse=False)
    labels_encoded = labels_encoded.reshape(len(labels_encoded), 1)
    labels_onehot = onehot_encoder.fit_transform(labels_encoded)
    return data, labels_onehot

target_frame_count = 30  # 목표 프레임 수 (예: 30프레임/초 동영상일 경우 1초)
target_height = 128
target_width = 128

# 데이터 불러오고 나누는 함수
def load_and_split_data(data_folder, test_size=0.2, random_state=42):
    # data 폴더 아래의 모든 라벨 폴더를 찾기
    label_folders = glob.glob(os.path.join(data_folder, '*\\'))
    
    file_paths = []
    labels = []
    print(label_folders)
    # 각 라벨 폴더에서 파일 경로와 레이블 수집
    for label_folder in label_folders:
        label = os.path.basename(os.path.normpath(label_folder))  # 라벨은 폴더 이름에서 추출
        files = glob.glob(os.path.join(label_folder, '*.avi'))
        file_paths.extend(files)
        labels.extend([label] * len(files))

    # 데이터를 무작위로 섞고 train-test split 수행
    train_file_paths, test_file_paths, train_labels, test_labels = train_test_split(
        file_paths, labels, test_size=test_size, random_state=random_state, stratify=labels
    )

    # 데이터 전처리 (load_and_preprocess_data 함수 사용)
    train_data, train_labels = load_and_preprocess_data(train_file_paths, train_labels, target_frame_count=30, target_width=128,target_height=128)
    test_data, test_labels = load_and_preprocess_data(test_file_paths, test_labels,target_frame_count=30, target_width=128,target_height=128)

    return train_data, train_labels, test_data, test_labels
# 3D CNN 모델 구축 함수
def build_3d_cnn_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
        tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu'),
        tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model


# 모델 학습 함수
def train_model(model, train_data, train_labels, num_epochs, batch_size):
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    history = model.fit(train_data, train_labels, epochs=num_epochs, batch_size=batch_size)
    return history

# 모델 평가 함수
def evaluate_model(model, test_data, test_labels):
    loss, accuracy = model.evaluate(test_data, test_labels)
    return loss, accuracy

# 데이터 불러오고 나누기
train_data, train_labels, test_data, test_labels = load_and_split_data(data_folder)

# 데이터 형태 조정 (채널 추가)
train_data = np.expand_dims(train_data, axis=-1)
test_data = np.expand_dims(test_data, axis=-1)
print(train_data.shape)
print(test_data.shape)

train_data_tf = tf.convert_to_tensor(train_data, dtype=tf.float32)
train_labels_tf = tf.convert_to_tensor(train_labels, dtype=tf.float32)

# 모델 구축
# input_shape = train_data.shape[1:]  # (frames, height, width, channels)
input_shape = (target_frame_count, target_height, target_width, 1)
num_classes = len(np.unique(train_labels))
# model = build_3d_cnn_model(input_shape, num_classes)
model = build_3d_cnn_model(input_shape, num_classes)

# 모델 학습
num_epochs = 10
batch_size = 16
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
history = model.fit(train_data_tf, train_labels_tf, epochs=num_epochs, batch_size=batch_size)

# 테스트 데이터를 텐서로 변환
test_data_tf = tf.convert_to_tensor(test_data, dtype=tf.float32)
test_labels_tf = tf.convert_to_tensor(test_labels, dtype=tf.float32)

# 모델 평가
test_loss, test_accuracy = model.evaluate(test_data_tf, test_labels_tf)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')



['C:\\PlayData\\sign_remaster\\Sign_Language_Remaster\\data\\bread_house\\', 'C:\\PlayData\\sign_remaster\\Sign_Language_Remaster\\data\\hello\\']


c:\Users\2580j\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\2580j\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(16, 30, 128, 128, 3, 1)
(4, 30, 128, 128, 3, 1)
Epoch 1/10


ValueError: in user code:

    File "c:\Users\2580j\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\2580j\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\2580j\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\2580j\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\2580j\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\2580j\App

In [55]:
def load_and_preprocess_data(folder_path, image_size=(128, 128), test_size=0.2, random_seed=42):
    labels = []
    data = []

    for label, subfolder in enumerate(os.listdir(folder_path)):
        subfolder_path = os.path.join(folder_path, subfolder)
        if os.path.isdir(subfolder_path):
            for filename in os.listdir(subfolder_path):
                if filename.endswith(".avi"):
                    video_path = os.path.join(subfolder_path, filename)
                    cap = cv2.VideoCapture(video_path)
                    frames = []

                    while cap.isOpened():
                        ret, frame = cap.read()
                        if not ret:
                            break
                        resized_frame = cv2.resize(frame, image_size)
                        frames.append(resized_frame)

                    frames = np.array(frames)
                    # 각 프레임을 여기서 처리하거나 사용하십시오.

                    labels.append(label)
                    data.append(frames)

                    cap.release()

    # 라벨링 및 데이터 수집 완료 후, train/test 데이터 분할 코드 추가
    data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=test_size, random_state=random_seed)
    # print("Train data shape:", train_data.shape)
    # print("Train labels shape:", train_labels.shape)
    # print("Test data shape:", test_data.shape)
    # print("Test labels shape:", test_labels.shape)
    # 데이터를 훈련 세트와 테스트 세트로 분할
    print("Train data count:", len(data_train))
    print("Test data count:", len(data_test))

    return data_train, data_test, labels_train, labels_test

# 사용 예시
folder_path = r"C:\PlayData\sign_remaster\Sign_Language_Remaster\data"  # AVI 파일이 있는 폴더 경로 입력
train_data, test_data, train_labels, test_labels = load_and_preprocess_data(folder_path)


Train data count: 16
Test data count: 4


In [2]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split

def load_and_preprocess_data(folder_path, target_frame_count=90, image_size=(128, 128), test_size=0.2, random_seed=42):
    labels = []
    data = []

    for label, subfolder in enumerate(os.listdir(folder_path)):
        subfolder_path = os.path.join(folder_path, subfolder)
        if os.path.isdir(subfolder_path):
            for filename in os.listdir(subfolder_path):
                if filename.endswith(".avi"):
                    video_path = os.path.join(subfolder_path, filename)
                    cap = cv2.VideoCapture(video_path)
                    frames = []

                    while cap.isOpened() and len(frames) < target_frame_count:
                        ret, frame = cap.read()
                        if not ret:
                            break
                        resized_frame = cv2.resize(frame, image_size)
                        frames.append(resized_frame)

                    # 프레임 수가 부족하면 패딩을 추가
                    while len(frames) < target_frame_count:
                        frames.append(np.zeros_like(frames[0]))

                    frames = np.array(frames)
                    # 여기서 frames를 사용하여 모델에 데이터로 사용할 수 있음

                    labels.append(label)
                    data.append(frames)

                    cap.release()

    # 데이터를 4D 텐서로 변환 (샘플 수, 프레임 수, 높이, 너비)
    data = np.array(data)

    # 데이터를 0~1 사이의 값으로 정규화
    data = data.astype('float32') / 255.0

    # 라벨을 원-핫 인코딩으로 변환
    labels = np.array(labels)
    num_classes = len(os.listdir(folder_path))
    labels = np.eye(num_classes)[labels]

    # 데이터를 훈련 세트와 테스트 세트로 분할
    train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=test_size, random_state=random_seed)

    print("Train data shape:", train_data.shape)
    print("Train labels shape:", train_labels.shape)
    print("Test data shape:", test_data.shape)
    print("Test labels shape:", test_labels.shape)

    return train_data, test_data, train_labels, test_labels

# 사용 예시
folder_path = r"C:\PlayData\sign_remaster\Sign_Language_Remaster\data"  # AVI 파일이 있는 폴더 경로 입력
train_data, test_data, train_labels, test_labels = load_and_preprocess_data(folder_path)


Train data shape: (16, 90, 128, 128, 3)
Train labels shape: (16, 2)
Test data shape: (4, 90, 128, 128, 3)
Test labels shape: (4, 2)


In [62]:
for i in range(len(train_data)):
    print(train_data[i].shape)

(90, 128, 128, 3)
(90, 128, 128, 3)
(90, 128, 128, 3)
(90, 128, 128, 3)
(90, 128, 128, 3)
(90, 128, 128, 3)
(90, 128, 128, 3)
(90, 128, 128, 3)
(90, 128, 128, 3)
(90, 128, 128, 3)
(90, 128, 128, 3)
(90, 128, 128, 3)
(90, 128, 128, 3)
(90, 128, 128, 3)
(90, 128, 128, 3)
(90, 128, 128, 3)


In [66]:
### 전처리된 파일 재생
import cv2

# 비디오 재생을 위한 함수 정의
def play_video(frames):
    for frame in frames:
        cv2.imshow("Video", frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):  # 'q' 키를 누르면 종료
            break

    cv2.destroyAllWindows()

# train_data 중 첫 번째 비디오 재생
for i in range(len(train_data)):
    play_video(train_data[i])


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
def create_3d_cnn_model(input_shape, num_classes):
    model = models.Sequential()

    # 3D Convolution layers
    model.add(layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Conv3D(64, (3, 3, 3), activation='relu'))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Conv3D(64, (3, 3, 3), activation='relu'))
    model.add(layers.MaxPooling3D((2, 2, 2)))

    # Fully connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model
# 모델 생성
input_shape = train_data.shape[1:]
num_classes = len(os.listdir(folder_path))
model = create_3d_cnn_model(input_shape, num_classes)

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
history = model.fit(train_data, train_labels, epochs=3, batch_size=4, validation_split=0.2)

# 모델 평가
test_loss, test_acc = model.evaluate(test_data, test_labels)
print(f'Test accuracy: {test_acc}')


Epoch 1/3
3/3 [==============================] - 50s 16s/step - loss: 0.6413 - accuracy: 0.7500 - val_loss: 0.5748 - val_accuracy: 0.7500
Epoch 2/3
3/3 [==============================] - 49s 17s/step - loss: 0.1891 - accuracy: 1.0000 - val_loss: 1.0867 - val_accuracy: 0.7500
Epoch 3/3
1/1 [==============================] - 1s 1s/step - loss: 1.8477e-06 - accuracy: 1.0000
Test accuracy: 1.0


In [4]:
# 모델을 저장할 경로 
model_save_path = 'C:/PlayData/sign_remaster/Sign_Language_Remaster/model/test10.h5'

# 모델 저장
model.save(model_save_path)

print(f'Model saved to {model_save_path}')


Model saved to C:/PlayData/sign_remaster/Sign_Language_Remaster/model/test10.h5


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
num = 0
cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = None
recording = False

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    if not ret:
        continue

    # 손 감지 및 랜드마크 추출
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # 검은색 배경 생성
    hand_landmarks_image = np.zeros_like(frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 손 랜드마크를 화면에 표시
            mp_drawing.draw_landmarks(hand_landmarks_image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                                      landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2),
                                      connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2))

    cv2.imshow('Video', hand_landmarks_image)

    # 여기서 이미지 전처리(입력크기는 90,128,128,3)
    # 모델에 90프레임을 입력하도록 처리
    pred = model.predict(hand_landmarks_image)
    # 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [12]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

# 모델 로드
# model = load_model("your_model_path.h5")  # 모델 파일 경로를 입력하세요

num = 0
cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 9개의 빈 배열 초기화
video_frames = [np.zeros((128, 128, 3), dtype=np.float32) for _ in range(9)]
frame_count = 0  # 현재까지 저장된 프레임 수

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    if not ret:
        continue

    # 손 감지 및 랜드마크 추출
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # 검은색 배경 생성
    hand_landmarks_image = np.zeros_like(frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 손 랜드마크를 화면에 표시
            mp_drawing.draw_landmarks(hand_landmarks_image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                                      landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2),
                                      connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2))

    # 이미지를 전처리하고 비디오 프레임 배열에 추가
    resized_image = cv2.resize(hand_landmarks_image, (128, 128))  # 입력 크기에 맞게 조정
    resized_image = resized_image / 255.0  # 정규화

    # 현재 프레임을 적절한 배열에 추가
    if frame_count % 10 == 0 and frame_count > 0:
        target_index = (frame_count // 10 - 1) % 9
        video_frames[target_index] = resized_image

    frame_count += 1

    # 90프레임이 모였을 때 모델에 전달하여 예측
    if frame_count >= 90:
        input_video = np.array(video_frames)  # 90프레임을 4D 배열로 변환
        input_video = np.expand_dims(input_video, axis=0)  # 배치 차원 추가

        # 모델에 예측 요청
        pred = model.predict(input_video)  # 모델에 예측 요청
        # 여기서 예측값(pred)를 사용할 수 있습니다.
        print(pred)

        # 프레임 카운트 및 비디오 프레임 배열 초기화
        frame_count = 0
        video_frames = [np.zeros((128, 128, 3), dtype=np.float32) for _ in range(9)]

    cv2.imshow('Video', hand_landmarks_image)

    # 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


InvalidArgumentError: Graph execution error:

Detected at node 'sequential/conv3d_2/Conv3D' defined at (most recent call last):
    File "c:\Users\2580j\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\2580j\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\2580j\AppData\Roaming\Python\Python38\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\2580j\AppData\Roaming\Python\Python38\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\2580j\AppData\Roaming\Python\Python38\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\2580j\AppData\Roaming\Python\Python38\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asynci

In [13]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 88, 126, 126, 32)  2624      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 44, 63, 63, 32)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 42, 61, 61, 64)    55360     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 21, 30, 30, 64)   0         
 3D)                                                             
                                                                 
 conv3d_2 (Conv3D)           (None, 19, 28, 28, 64)    110656    
                                                                 
 max_pooling3d_2 (MaxPooling  (None, 9, 14, 14, 64)    0

In [19]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

# 모델 로드
# model = load_model("your_model_path.h5") 

num = 0
cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 88프레임을 담을 빈 배열 초기화
video_frames = [np.zeros((128, 128, 3), dtype=np.float32) for _ in range(88)]
frame_count = 0  # 현재까지 저장된 프레임 수

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    if not ret:
        continue

    # 손 감지 및 랜드마크 추출
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # 검은색 배경 생성
    hand_landmarks_image = np.zeros_like(frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 손 랜드마크를 화면에 표시
            mp_drawing.draw_landmarks(hand_landmarks_image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                                      landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2),
                                      connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2))

    # 이미지를 전처리하고 비디오 프레임 배열에 추가
    resized_image = cv2.resize(hand_landmarks_image, (128, 128))  # 입력 크기에 맞게 조정
    resized_image = resized_image / 255.0  # 정규화

    # 현재 프레임을 빈 배열 중 적절한 위치에 추가
    target_index = frame_count % 88
    video_frames[target_index] = resized_image
    frame_count += 1

    # 88프레임이 모였을 때 모델에 전달하여 예측
    if frame_count >= 88:
        input_video = np.array(video_frames)  # 88프레임을 4D 배열로 변환
        input_video = np.expand_dims(input_video, axis=0)  # 배치 차원 추가

        # 모델에 예측 요청
        pred = model.predict(input_video)  # 모델에 예측 요청

        # 예측된 클래스를 확인하고 라벨 출력
        predicted_class = np.argmax(pred, axis=1)  # 확률이 가장 높은 클래스의 인덱스 가져오기
        if predicted_class == 0:
            print("Predicted Label: house")
        else:
            print("Predicted Label: bread_house")

    cv2.imshow('Video', hand_landmarks_image)

    # 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 294ms/step
Predicted Label: bread_house
1/1 [==============================] - 0s 297ms/step
Predicted Label: bread_house
1/1 [==============================] - 0s 322ms/step
Predicted Label: bread_house
1/1 [==============================] - 0s 293ms/step
Predicted Label: bread_house
1/1 [==============================] - 0s 299ms/step
Predicted Label: bread_house
1/1 [==============================] - 0s 311ms/step
Predicted Label: bread_house
1/1 [==============================] - 0s 296ms/step
Predicted Label: bread_house
1/1 [==============================] - 0s 268ms/step
Predicted Label: bread_house
1/1 [==============================] - 0s 258ms/step
Predicted Label: bread_house
1/1 [==============================] - 0s 292ms/step
Predicted Label: bread_house
1/1 [==============================] - 0s 453ms/step
Predicted Label: bread_house
1/1 [==============================] - 0s 261ms/step
Predicted Label: bread_house


In [22]:
video_frames = [np.zeros((128, 128, 3), dtype=np.float32) for _ in range(88)]
video_frames[0].shape

(128, 128, 3)